In [ ]:
from sklearn.neighbors import KNeighborsClassifier # classificadores utilizados
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import train_test_split # realização da separação do dataset
from sklearn.metrics import classification_report # valores de recall e f1-score
from imutils import paths
import numpy as np
import argparse
import cv2
import os
import imutils
import time

In [ ]:
inicio = time.time() # para realizar o cálculo do tempo 

In [ ]:
def img_para_raw(imagem, size=(128, 128)):
    # reescala a imagem para um tamanho fixo e gera uma lista de 
    # intensidades de raw pixel em um array
    return cv2.resize(imagem, size).flatten()

In [ ]:
def extrair_histograma(image, bins=(8, 8, 8)):
    # extrai um histograma de cores de 3 dimensões do espaço HSV
    # utilizando a quantidade de bins fornecida 
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
                        [0, 180, 0, 256, 0, 256])
    
    # realiza a normalização da imagem
    cv2.normalize(hist, hist) # alargamento de contraste

    return hist.flatten()

In [ ]:
# define os argumentos utilizados para execução do programa
arg = argparse.ArgumentParser()
arg.add_argument("-d", "--dataset", required=True,
	help="caminho para o dataset")
arg.add_argument("-k", "--vizinhos", type=int, default=1,
	help="# de vizinhos mais proximos para classificacao")
args = vars(arg.parse_args())

In [ ]:
# pega as imagens do argumento do dataset para um array
print("Processando imagens...")
imagePaths = list(paths.list_images(args["dataset"]))
# inicializa os arrays a serem utilizados para preenchimento
rawImages = []
features = []
labels = []

In [ ]:
for (i, imagePath) in enumerate(imagePaths):
    # carrega a imagem e extrai seu rótulo de classe para o array label
    image = cv2.imread(imagePath)
    label = imagePath.split(os.path.sep)[-1].split(".")[0]

    # utiliza os algoritmos nas imagens
    pixels = img_para_raw(image)
    hist = extrair_histograma(image)

    # atualiza as matrizes com os valores
    rawImages.append(pixels)
    features.append(hist)
    labels.append(label)



    # mostra uma atualização a cada 1000 imagens
    if i > 0 and i % 1000 == 0:
        print("[INFO] processados {}/{}".format(i, len(imagePaths)))

In [ ]:
processado = time.time()
print(processado - inicio) # mostra o tempo levado na extração de características

In [ ]:
# mostra informações dos vetores na memória
# transforma os arrays em array do NumPy
rawImages = np.array(rawImages) # mais rápidos
features = np.array(features) # maior performance
labels = np.array(labels) # maior funcionalidade
print("Matriz de pixels: {:.2f}MB".format(
	rawImages.nbytes / (1024 * 1000.0)))
print("Matriz de caracteristicas: {:.2f}MB".format(
	features.nbytes / (1024 * 1000.0)))

In [ ]:
# separa os dados do dataset em 75% para treinamento e 25% para testes
(trainRI, testRI, trainRL, testRL) = train_test_split(
	rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
	features, labels, test_size=0.25, random_state=42)

In [ ]:
# treinamento e avaliação do classificador k-NN em raw pixel
print("Avaliando precisao em raw pixel...")
modelo = KNeighborsClassifier(n_neighbors=args["neighbors"],n_jobs=-1) # o -1 indica a utilização de todos os cores nos jobs
modelo.fit(trainRI, trainRL)
acc = modelo.score(testRI, testRL)
print("Precisao em raw pixel: {:.2f}%".format(acc * 100))

y_pred = model.predict(testRI)
print(classification_report(testRL,y_pred)) # relatorio completo

In [ ]:
# treinamento e avaliação do classificador k-NN em histograma
print("Avaliando precisao em histograma...")
modelo = KNeighborsClassifier(n_neighbors=args["neighbors"],n_jobs=-1)
modelo.fit(trainFeat, trainLabels)
acc = modelo.score(testFeat, testLabels)
print("Precisao em histograma: {:.2f}%".format(acc * 100))

y_pred = model.predict(testFeat)
print(classification_report(testLabels,y_pred)) # relatorio completo

In [ ]:
# Geração da Rede Neural
modelo = MLPClassifier(solver='adam', #utilização do classificador adam
                    learning_rate='constant', #taxa de aprendizagem constante e a mesma para todos os neurônios
                    learning_rate_init=1e-3, #aprendizagem inicial de 0.001
                    alpha=1e-5, # valor de regularização L2
                    hidden_layer_sizes=(5, 2), # definição das camadas escondidas
                    max_iter=200, # qtde de epochs
                    random_state=1)

# Treinamento do modelo
modelo.fit(trainFeat, trainLabels)

# Previsao utilizando o dataset de testes
y_pred = modelo.predict(testFeat)


print("Precisao:", metrics.accuracy_score(testLabels, y_pred))

print("Classificacao %s:\n%s\n"
      % (modelo, metrics.classification_report(testLabels, y_pred)))


In [ ]:
# Mapeamento das Máquinas de Vetores de Suporte
modelo = svm.SVC(gamma=0.001, # valor de gamma 0.001
                tol=1e-3) # tolerâncica 0.001

# Treinamento do modelo
modelo.fit(trainFeat, trainLabels)

# Previsao utilizando o dataset de testes
y_pred = modelo.predict(testFeat)

In [ ]:
# Criação da Árvore de decisões
modelo = tree.DecisionTreeClassifier(min_samples_split=2, # minimo necessario para se dividir um nó interno
                                    max_depth=None, # se expandir até ficar pura
                                    min_samples_leaf=1) # não divide se não deixar no mínimo uma folha de cada lado

#Treinamento do modelo
modelo.fit(trainFeat, trainLabels)

#Previsao utilizando o dataset de testes
y_pred = modelo.predict(testFeat)
